In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# Setting random seeds to reduce the amount of randomness in the neural net weights and results.
# The results may still not be exactly reproducible.
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
# Checking the installed version of TensorFlow.
# The code in this notebook was written using TensorFlow version 2.2.0.

tf.__version__

'2.0.0'

After understanding the effects of all the improvement and optimization techniques on the CNN model performance, **the final CNN model can now be defined and trained**:
1. with **grayscale images** instead of RGB coloured images.
2. for **60 epochs**.
3. for **re-distributed classes of age-ranges**.
4. with an **optimized architecture**, comprising of:
- an input *Conv2D* layer (with 32 filters) paired with an *AveragePooling2D* layer,
- 3 pairs of *Conv2D* (with 64, 128 & 256 filters) and *AveragePooling2D* layers,
- a *GlobalAveragePooling2D* layer,
- 1 *Dense* layer with 132 nodes, and
- an output *Dense* layer with 7 nodes.

In [3]:
# Importing the training dataset and testing dataset to create tensors of images using the filename paths.


train_df = pd.read_csv("data/images_filenames_labels_train.csv")
test_df = pd.read_csv("data/images_filenames_labels_test.csv")

In [4]:
# Dropping the age column since classes of age-ranges have been re-distributed from 11 to 7 classes.

train_df.drop(columns=['target'], inplace=True)
train_df.head()

,filename,age
0,content/combined_faces/18_369.jpg,18
1,content/combined_faces/5_42.jpg,5
2,content/combined_faces/5_204.jpg,5
3,content/combined_faces/39_24.jpg,39
4,content/combined_faces/22_41.jpg,22


In [5]:
# Dropping the age column since classes of age-ranges have been re-distributed from 11 to 7 classes.

test_df.drop(columns=['target'], inplace=True)
test_df.head()

,filename,age
0,content/combined_faces/8_163.jpg,8
1,content/combined_faces/38_66.jpg,38
2,content/combined_faces/40_177.jpg,40
3,content/combined_faces/36_267.jpg,36
4,content/combined_faces/8_349.jpg,8


In [6]:
# Defining a function to return the class labels corresponding to the re-distributed 7 age-ranges.

def class_labels_reassign(age):

    if 1 <= age <= 2:
        return 0
    elif 3 <= age <= 9:
        return 1
    elif 10 <= age <= 20:
        return 2
    elif 21 <= age <= 27:
        return 3
    elif 28 <= age <= 45:
        return 4
    elif 46 <= age <= 65:
        return 5
    else:
        return 6

In [7]:
train_df['target'] = train_df['age'].map(class_labels_reassign)
test_df['target'] = test_df['age'].map(class_labels_reassign)

In [8]:
train_df['target'].value_counts(normalize=True)

4    0.279394
3    0.199829
5    0.167193
0    0.095307
2    0.093643
1    0.084087
6    0.080546
Name: target, dtype: float64

In [9]:
test_df['target'].value_counts(normalize=True)

4    0.279415
3    0.199781
5    0.167131
0    0.095361
2    0.093669
1    0.084113
6    0.080530
Name: target, dtype: float64

In [10]:
# Converting the filenames and target class labels into lists for augmented train and test datasets.

train_filenames_list = list(train_df['filename'])
train_labels_list = list(train_df['target'])

test_filenames_list = list(test_df['filename'])
test_labels_list = list(test_df['target'])

In [11]:
# Creating tensorflow constants of filenames and labels for augmented train and test datasets from the lists defined above.

train_filenames_tensor = tf.constant(train_filenames_list)
train_labels_tensor = tf.constant(train_labels_list)

test_filenames_tensor = tf.constant(test_filenames_list)
test_labels_tensor = tf.constant(test_labels_list)

In [12]:
# Defining a function to read the image, decode the image from given tensor and one-hot encode the image label class.
# Changing the channels para in tf.io.decode_jpeg from 3 to 1 changes the output images from RGB coloured to grayscale.

num_classes = 7

def _parse_function(filename, label):
    
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=1)    # channels=1 to convert to grayscale, channels=3 to convert to RGB.
    # image_resized = tf.image.resize(image_decoded, [200, 200])
    label = tf.one_hot(label, num_classes)

    return image_decoded, label

In [13]:
# Getting the dataset ready for the neural network.
# Using the tensor vectors defined above, accessing the images in the dataset and passing them through the function defined above.

train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames_tensor, train_labels_tensor))
train_dataset = train_dataset.map(_parse_function)
# train_aug_dataset = train_aug_dataset.repeat(3)
train_dataset = train_dataset.batch(512)    # Same as batch_size hyperparameter in model.fit() below.

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames_tensor, test_labels_tensor))
test_dataset = test_dataset.map(_parse_function)
# test_dataset = test_dataset.repeat(3)
test_dataset = test_dataset.batch(512)    # Same as batch_size hyperparameter in model.fit() below.

In [14]:
# Defining the architecture of the sequential neural network.

final_cnn = Sequential()

# Input layer with 32 filters, followed by an AveragePooling2D layer.
final_cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(200, 200, 1)))    # 3rd dim = 1 for grayscale images.
final_cnn.add(AveragePooling2D(pool_size=(2,2)))

# Three Conv2D layers with filters increasing by a factor of 2 for every successive Conv2D layer.
final_cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))

final_cnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))

final_cnn.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))

# A GlobalAveragePooling2D layer before going into Dense layers below.
# GlobalAveragePooling2D layer gives no. of outputs equal to no. of filters in last Conv2D layer above (256).
final_cnn.add(GlobalAveragePooling2D())

# One Dense layer with 132 nodes so as to taper down the no. of nodes from no. of outputs of GlobalAveragePooling2D layer above towards no. of nodes in output layer below (7).
final_cnn.add(Dense(132, activation='relu'))

# Output layer with 7 nodes (equal to the no. of classes).
final_cnn.add(Dense(7, activation='softmax'))

final_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      320       
_________________________________________________________________
average_pooling2d (AveragePo (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
average_pooling2d_1 (Average (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 128)       73856     
_________________________________________________________________
average_pooling2d_2 (Average (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 256)       2

In [15]:
# Compiling the above created CNN architecture.

final_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Fitting the above created CNN model.

final_cnn_history = final_cnn.fit(train_dataset,
                                  validation_data=test_dataset,
                                  epochs=1,
                                  shuffle=False    # shuffle=False to reduce randomness and increase reproducibility
                                 )

46/46 [==============================] - 660s 14s/step - loss: 2.3570 - accuracy: 0.2608 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


## Task: improve base model

Consider the following improvements 
    1. Number of epochs
    2. Model archetecture 
    3. Overfitting 
    4. Topics not covered in this course (Transfer Learning and Data Augmentation)